In [1]:
import data_engine as de
import pandas as pd
from feature_engine.discretisation import EqualWidthDiscretiser
from feature_engine.imputation import MeanMedianImputer,CategoricalImputer

In [2]:
data_save_path = './data/'
root_path = r'C:\\Users\\8prab\\Documents\\NSE_Equity_Futures_iEOD\\'
pattern = '*BANKNIFTY.txt'
data_name = 'banknifty'
#base_df= de.create_dataset(root_path,pattern,data_save_path,data_name)
base_df = pd.read_csv(r'C:\Users\8prab\Google Drive\Work\trading\data\banknifty\banknifty.csv')

In [11]:
from sklearn.pipeline import Pipeline
pipe = Pipeline([
    ('nd', de.NormalizeDataset(columns = ['close','open','high','low'],impute_values=True,impute_type = 'mean_median_imputer',convert_to_floats = True)),
    ('labelgenerator_1', de.LabelCreator(freq='1min',shift=-15,shift_column='close')),
    ('labelgenerator_2', de.LabelCreator(freq='1min',shift=-30,shift_column='close')),
    ('tech_indicator', de.TechnicalIndicator()),
    ('nd2', de.NormalizeDataset(columns = ['close','open','high','low'],fillna=True,fillna_method='bfill')),
    ('nd3', de.NormalizeDataset(columns = ['close','open','high','low'],drop_na_rows=False,fillna=True,fillna_method='ffill')),
    ('ltgvc', de.LastTicksGreaterValuesCount(columns=['close','open'])),
    #('nd3', de.NormalizeDataset(columns = ['close','open','high','low'],fillna=True)),
    ('pltbc', de.PriceLastTickBreachCount(columns=['close','open']))
    ])
base_df = pipe.fit(base_df).transform(base_df)

Shape of dataframe after transform is (1105219, 141)
Shape of dataframe after transform is (1105219, 141)
Shape of dataframe after transform is (1105219, 141)
Function momentum_AROON was unable to run, Error is Too many price arguments: expected 2 (high, low)
Function momentum_AROONOSC was unable to run, Error is Too many price arguments: expected 2 (high, low)
Function momentum_MFI was unable to run, Error is 'Signals' object has no attribute 'volume'
Function volumn_AD was unable to run, Error is 'Signals' object has no attribute 'volume'
Function volumn_ADOSC was unable to run, Error is 'Signals' object has no attribute 'volume'
Function volumn_OBV was unable to run, Error is 'Signals' object has no attribute 'volume'
Function pattern_abondoned_baby was unable to run, Error is Invalid parameter value for penetration (expected float, got int)
Function pattern_dark_cloud_cover was unable to run, Error is Invalid parameter value for penetration (expected float, got int)
Function patter

c:\Users\8prab\Google Drive\Work\trading\data_engine.py:786: RuntimeWarning: invalid value encountered in greater
  df[f'last_tick_{col}_{self.last_ticks}']  = (arr[:, :-1] > arr[:, [-1]]).sum(axis=1)


Shape of dataframe after transform is (1105219, 143)


In [13]:
base_df['test'] = base_df['close'].rolling('10min', min_periods=1).apply(lambda x: (x[-1] > x[:-1]).mean()).astype(float)

In [43]:
t = base_df[0:20]
import numpy as np

In [66]:
t['test'] = t['close'].rolling('10min', min_periods=0).apply(lambda x: (x[-1] < x[:-1]).sum()).fillna(0)
t['test1'] = t['close'].rolling('10min', min_periods=0).apply(lambda x: x[:].mean()).fillna(0)
t['test2'] = t['close'].rolling('10min', min_periods=0).apply(lambda x: (x > x[:].mean()).sum()).fillna(0)
t['test3'] = t['close'].rolling('10min', min_periods=0).apply(lambda x: (x < x[:].mean()).sum()).fillna(0)
t['test4'] = t['close'].rolling('10min', min_periods=0).apply(lambda x: (x < x[:].quantile(0.25)).sum()).fillna(0)
t['test5'] = t['close'].rolling('10min', min_periods=0).apply(lambda x: x[:].quantile(0.25)).fillna(0)
t[['close','test','test1','test2','test3','test4','test5']]

<ipython-input-66-8723029f915a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t['test'] = t['close'].rolling('10min', min_periods=0).apply(lambda x: (x[-1] < x[:-1]).sum()).fillna(0)
<ipython-input-66-8723029f915a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t['test1'] = t['close'].rolling('10min', min_periods=0).apply(lambda x: x[:].mean()).fillna(0)
<ipython-input-66-8723029f915a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

,close,test,test1,test2,test3,test4,test5
2010-03-02 09:01:00,8839.80,0.0,8839.800000,0.0,0.0,0.0,8839.8000
2010-03-02 09:02:00,8852.30,0.0,8846.050000,1.0,1.0,1.0,8842.9250
2010-03-02 09:03:00,8846.65,1.0,8846.250000,2.0,1.0,1.0,8843.2250
2010-03-02 09:04:00,8846.70,1.0,8846.362500,3.0,1.0,1.0,8844.9375
2010-03-02 09:05:00,8841.60,3.0,8845.410000,3.0,2.0,1.0,8841.6000
2010-03-02 09:06:00,8836.90,5.0,8843.991667,3.0,3.0,2.0,8840.2500
2010-03-02 09:07:00,8846.35,3.0,8844.328571,4.0,3.0,2.0,8840.7000
2010-03-02 09:08:00,8855.70,0.0,8845.750000,5.0,3.0,2.0,8841.1500
2010-03-02 09:09:00,8859.15,0.0,8847.238889,3.0,6.0,2.0,8841.6000
2010-03-02 09:10:00,8864.95,0.0,8849.010000,4.0,6.0,3.0,8842.7875


In [16]:
base_df[['close','test']]

,close,test
2010-03-02 09:01:00,8839.80,NaN
2010-03-02 09:02:00,8852.30,1.000000
2010-03-02 09:03:00,8846.65,0.500000
2010-03-02 09:04:00,8846.70,0.666667
2010-03-02 09:05:00,8841.60,0.250000
...,...,...
2022-01-13 15:28:00,38467.20,0.444444
2022-01-13 15:29:00,38456.65,0.000000
2022-01-13 15:30:00,38465.70,0.222222
2022-01-13 15:31:00,38466.05,0.333333


In [10]:
for col in base_df.columns.tolist():
    print(col,len(base_df[col].isnull()),base_df[col].isna().sum())
#base_df.head()

open 1105219 0
high 1105219 0
low 1105219 0
close 1105219 0
label_-15_1min_close 1105219 0
label_-30_1min_close 1105219 0
momentum_ADX 1105219 0
momentum_ADXR 1105219 0
momentum_APO 1105219 0
momentum_BOP 1105219 0
momentum_CCI 1105219 0
momentum_CMO 1105219 0
momentum_DX 1105219 0
momentum_MACD 1105219 0
momentum_MACD_SIGNAL 1105219 0
momentum_MACD_HIST 1105219 0
momentum_MACDEXT 1105219 0
momentum_MACDEXT_SIGNAL 1105219 0
momentum_MACDEXT_HIST 1105219 0
momentum_MACDFIX 1105219 0
momentum_MACDFIX_SIGNAL 1105219 0
momentum_MACDFIX_HIST 1105219 0
momentum_MINUS_DM 1105219 0
momentum_MOM 1105219 0
momentum_PLUS_DI 1105219 0
momentum_PLUS_DM 1105219 0
momentum_PPO 1105219 0
momentum_ROC 1105219 0
momentum_ROCP 1105219 0
momentum_ROCR 1105219 0
momentum_ROCR100 1105219 0
momentum_RSI 1105219 0
momentum_SLOWK 1105219 0
momentum_SLOWD 1105219 0
momentum_FASTK 1105219 0
momentum_FASTD 1105219 0
momentum_STOCHRSI_FASTK 1105219 0
momentum_STOCHRSI_FASTD 1105219 0
momentum_TRIX 1105219 0
moment

In [9]:
base_df.dtypes

open                                 float64
high                                 float64
low                                  float64
close                                float64
label_-15_1min_close                  object
                                      ...   
overlap_midpoint                     float64
overlap_midpoint_price               float64
overlap_simple_moving_avg            float64
overlap_triangular_moving_average    float64
overlap_weighted_moving_average      float64
Length: 141, dtype: object

In [6]:
from sklearn.pipeline import Pipeline
pipe = Pipeline([
    ('nd', de.NormalizeDataset(columns = ['close','open','high','low'],impute_values=True,impute_type = 'mean_median_imputer',convert_to_floats = True,arbitrary_impute_variable=99)),
    ('labelgenerator_1', de.LabelCreator(freq='1min',shift=-15,shift_column='close')),
    ('labelgenerator_2', de.LabelCreator(freq='1min',shift=-30,shift_column='close')),
    ('tech_indicator', de.TechnicalIndicator()),
    ('nd2', de.NormalizeDataset(columns = ['close','open','high','low'],impute_values=False,convert_to_floats = False,drop_na_rows=True)),
    ('ltgvc', de.LastTicksGreaterValuesCount(columns=['close','open'])),
   # ('pltbc', de.PriceLastTickBreachCount(columns=['close','open'])),
   # ('pdrhw', de.PriceDayRangeHourWise()),
   # ('pv', de.PriceVelocity()),
   # ('ppi', de.PricePerIncrement()),
    ])
base_df = pipe.fit(base_df).transform(base_df)


Function momentum_AROON was unable to run, Error is Too many price arguments: expected 2 (high, low)
Function momentum_AROONOSC was unable to run, Error is Too many price arguments: expected 2 (high, low)
Function momentum_MFI was unable to run, Error is 'Signals' object has no attribute 'volume'
Function volumn_AD was unable to run, Error is 'Signals' object has no attribute 'volume'
Function volumn_ADOSC was unable to run, Error is 'Signals' object has no attribute 'volume'
Function volumn_OBV was unable to run, Error is 'Signals' object has no attribute 'volume'
Function pattern_abondoned_baby was unable to run, Error is Invalid parameter value for penetration (expected float, got int)
Function pattern_dark_cloud_cover was unable to run, Error is Invalid parameter value for penetration (expected float, got int)
Function pattern_morning_doji_star was unable to run, Error is Invalid parameter value for penetration (expected float, got int)
Function pattern_morning_star was unable to r

KeyboardInterrupt: 

In [8]:
base_df.shape

(0, 143)

In [8]:
type(base_df.index)

pandas.core.indexes.datetimes.DatetimeIndex

In [9]:
obj = EqualWidthDiscretiser(variables=['open'], bins=10, return_object=False, return_boundaries=False)
obj.fit(base_df.fillna(0)).transform(base_df)

,open,high,low,close,label_-15_1min_close,label_-30_1min_close
2010-03-02 09:01:00,1,8856.95,8826.90,8839.80,40to60,40to60
2010-03-02 09:02:00,1,8853.90,8832.80,8852.30,40to60,20to40
2010-03-02 09:03:00,1,8855.70,8844.65,8846.65,40to60,40to60
2010-03-02 09:04:00,1,8850.70,8843.75,8846.70,40to60,20to40
2010-03-02 09:05:00,1,8846.40,8837.75,8841.60,60to80,40to60
...,...,...,...,...,...,...
2022-01-13 15:28:00,9,38469.15,38459.45,38467.20,unknown,unknown
2022-01-13 15:29:00,9,38479.85,38456.65,38456.65,unknown,unknown
2022-01-13 15:30:00,9,38471.55,38455.15,38465.70,unknown,unknown
2022-01-13 15:31:00,9,38466.05,38466.05,38466.05,unknown,unknown


In [8]:
pipe.get_params()

{'memory': None,
 'steps': [('labelgenerator_1', LabelCreator()),
  ('labelgenerator_2', LabelCreator(shift=-30)),
  ('technical', TechnicalIndicator())],
 'verbose': False,
 'labelgenerator_1': LabelCreator(),
 'labelgenerator_2': LabelCreator(shift=-30),
 'technical': TechnicalIndicator(),
 'labelgenerator_1__freq': '1min',
 'labelgenerator_1__shift': -15,
 'labelgenerator_1__shift_column': 'close',
 'labelgenerator_2__freq': '1min',
 'labelgenerator_2__shift': -30,
 'labelgenerator_2__shift_column': 'close'}

In [9]:
pipe.set_params()

Pipeline(steps=[('labelgenerator_1', LabelCreator()),
                ('labelgenerator_2', LabelCreator(shift=-30)),
                ('technical', TechnicalIndicator())])

In [10]:
from feature_engine.discretisation import EqualWidthDiscretiser
from feature_engine.imputation import MeanMedianImputer

In [11]:
base_df.columns

Index(['open', 'high', 'low', 'close', 'label_-15_1min_close',
       'label_-30_1min_close', 'momentum_ADX', 'momentum_ADXR', 'momentum_APO',
       'momentum_BOP',
       ...
       'overlap_double_exp_moving_avg', 'overlap_exp_moving_avg',
       'overlap_hilbert_transform', 'overlap_kaufman_adaptive_moving_avg',
       'overlap_moving_avg', 'overlap_midpoint', 'overlap_midpoint_price',
       'overlap_simple_moving_avg', 'overlap_triangular_moving_average',
       'overlap_weighted_moving_average'],
      dtype='object', length=141)

In [5]:
n = 5
base_df['close'] = (base_df['close'].rolling(n+1, min_periods=1).apply(lambda x: (x[-1] > x[:-1]).mean()).astype(int))

In [6]:
import numpy as np

In [7]:
n = 2
x = np.concatenate([[np.nan] * (n), base_df['close'].values])

def rolling_window(a, window):
    shape = a.shape[:-1] + (a.shape[-1] - window + 1, window)
    strides = a.strides + (a.strides[-1],)
    return np.lib.stride_tricks.as_strided(a, shape=shape, strides=strides)
arr = rolling_window(x, n + 1)

base_df['new'] = (arr[:, :-1] > arr[:, [-1]]).sum(axis=1)

<ipython-input-7-402c5888b12e>:10: RuntimeWarning: invalid value encountered in greater
  base_df['new'] = (arr[:, :-1] > arr[:, [-1]]).sum(axis=1)


In [8]:
base_df

,open,high,low,close,new
2018-04-02 09:08:00,24235.70,24235.70,24235.70,24235.70,0
2018-04-02 09:16:00,24268.00,24300.60,24243.70,24253.10,0
2018-04-02 09:17:00,24257.40,24286.10,24219.30,24221.40,2
2018-04-02 09:18:00,24222.60,24224.40,24204.00,24220.70,2
2018-04-02 09:19:00,24218.70,24247.10,24218.70,24229.40,0
...,...,...,...,...,...
2011-12-30 15:26:00,7955.80,7956.25,7954.30,7956.25,1
2011-12-30 15:27:00,7955.80,7959.90,7955.80,7959.80,0
2011-12-30 15:28:00,7960.90,7963.45,7960.90,7961.40,0
2011-12-30 15:29:00,7959.45,7959.45,7953.30,7954.70,2


In [31]:
base_df

,open,high,low,close,label_-15_1min_close,label_-30_1min_close,new
2010-03-02 09:01:00,8826.90,8856.95,8826.90,8839.80,40to60,40to60,0
2010-03-02 09:02:00,8840.75,8853.90,8832.80,8852.30,40to60,20to40,0
2010-03-02 09:03:00,8855.70,8855.70,8844.65,8846.65,40to60,40to60,1
2010-03-02 09:04:00,8843.85,8850.70,8843.75,8846.70,40to60,20to40,1
2010-03-02 09:05:00,8846.40,8846.40,8837.75,8841.60,60to80,40to60,2
...,...,...,...,...,...,...,...
2022-01-13 15:28:00,38464.35,38469.15,38459.45,38467.20,unknown,unknown,1
2022-01-13 15:29:00,38469.00,38479.85,38456.65,38456.65,unknown,unknown,2
2022-01-13 15:30:00,38464.05,38471.55,38455.15,38465.70,unknown,unknown,1
2022-01-13 15:31:00,38466.05,38466.05,38466.05,38466.05,unknown,unknown,0


In [34]:
last_ticks = 2
base_df = base_df.replace([np.inf, -np.inf], np.nan).dropna(subset=["close"], how="all")
base_df['new1'] = (base_df['close'].rolling(last_ticks+1, min_periods=1)
                        .apply(lambda x: (x[-1] > x[:-1]).mean())
                        .astype(int))
base_df

ValueError: Cannot convert non-finite values (NA or inf) to integer

In [25]:
import numpy as np
df = pd.DataFrame({"a": np.random.randint(1, high=10, size=50)})
#print(df)
def f(s):
    #d = s.to_frame().groupby(pd.cut(s, np.arange(s.min(), s.max()*1.01, 1), include_lowest=True)).transform("count")
    tmp, bins = pd.cut(
                x=s, bins=3, retbins=True, duplicates="drop"
            )
    bins = list(bins)
    bins[0] = float("-inf")
    bins[len(bins) - 1] = float("inf")
    return pd.cut(x=s, bins=bins, labels=False)

df["a"].rolling(10).apply(f)

TypeError: cannot convert the series to <class 'float'>

In [21]:
tmp, bins = pd.cut(
                x=df['a'], bins=3, retbins=True, duplicates="drop"
            )
bins

array([0.992     , 3.66666667, 6.33333333, 9.        ])

In [22]:
bins = list(bins)
bins[0] = float("-inf")
bins[len(bins) - 1] = float("inf")
bins

[-inf, 3.6666666666666665, 6.333333333333333, inf]

In [24]:
t = pd.cut(x=df['a'], bins=bins, labels=False)
t

0     0
1     2
2     2
3     2
4     0
5     1
6     2
7     2
8     1
9     2
10    0
11    0
12    2
13    2
14    0
15    1
16    1
17    1
18    1
19    0
20    1
21    0
22    0
23    2
24    1
25    1
26    2
27    2
28    1
29    1
30    2
31    0
32    0
33    2
34    2
35    1
36    0
37    1
38    2
39    0
40    1
41    1
42    0
43    0
44    1
45    0
46    1
47    2
48    1
49    0
Name: a, dtype: int64

In [30]:
i = pd.date_range('2018-04-09', periods=100, freq='30min')
ts = pd.DataFrame({'A': list(range(100))}, index=i)
ts

,A
2018-04-09 00:00:00,0
2018-04-09 00:30:00,1
2018-04-09 01:00:00,2
2018-04-09 01:30:00,3
2018-04-09 02:00:00,4
...,...
2018-04-10 23:30:00,95
2018-04-11 00:00:00,96
2018-04-11 00:30:00,97
2018-04-11 01:00:00,98


In [32]:
ts.between_time('09:00', '10:30')['A']

2018-04-09 09:00:00    18
2018-04-09 09:30:00    19
2018-04-09 10:00:00    20
2018-04-09 10:30:00    21
2018-04-10 09:00:00    66
2018-04-10 09:30:00    67
2018-04-10 10:00:00    68
2018-04-10 10:30:00    69
Name: A, dtype: int64

In [63]:
from datetime import datetime


In [83]:
s1 = ts.between_time('09:00', '10:30').groupby(pd.Grouper(freq='d')).max() - ts.between_time('09:00', '10:30').groupby(pd.Grouper(freq='d')).min()
s1 = ts['A'].between_time('09:00', '10:30').groupby(pd.Grouper(freq='d')).max() 
#s1.index  =str(s1.index) + '10:30:00'	
s1.index = pd.to_datetime(s1.index) 
s1 = s1.sort_index()
s1.index = s1.index + pd.DateOffset(minutes=60*10.5)
s1

2018-04-09 10:30:00    21
2018-04-10 10:30:00    69
Name: A, dtype: int64

In [71]:
s1.index

DatetimeIndex(['2018-04-09', '2018-04-10'], dtype='datetime64[ns]', freq='D')

In [42]:
s1.dtypes

A    int64
dtype: object

In [84]:
merge=pd.merge(ts,s1, how='outer', left_index=True, right_index=True)
merge = merge.fillna(method='ffill')
#merge

In [85]:
merge.between_time('09:00', '12:30')

,A_x,A_y
2018-04-09 09:00:00,18,NaN
2018-04-09 09:30:00,19,NaN
2018-04-09 10:00:00,20,NaN
2018-04-09 10:30:00,21,21.0
2018-04-09 11:00:00,22,21.0
2018-04-09 11:30:00,23,21.0
2018-04-09 12:00:00,24,21.0
2018-04-09 12:30:00,25,21.0
2018-04-10 09:00:00,66,21.0
2018-04-10 09:30:00,67,21.0


In [89]:
base_df.head()

,open,high,low,close
20180402 09:08,24235.7,24235.7,24235.7,24235.7
20180402 09:16,24268.0,24300.6,24243.7,24253.1
20180402 09:17,24257.4,24286.1,24219.3,24221.4
20180402 09:18,24222.6,24224.4,24204.0,24220.7
20180402 09:19,24218.7,24247.1,24218.7,24229.4


In [92]:
base_df.index = pd.to_datetime(base_df.index)
base_df = base_df.sort_index()
base_df = base_df[~base_df.index.duplicated(keep='first')]  
s1 = base_df['high'].between_time('09:00', '10:30').groupby(pd.Grouper(freq='d')).max() - base_df['low'].between_time('09:00', '10:30').groupby(pd.Grouper(freq='d')).min()
#s1 = ts['A'].between_time('09:00', '10:30').groupby(pd.Grouper(freq='d')).max() 
#s1.index  =str(s1.index) + '10:30:00'	
s1.index = pd.to_datetime(s1.index) 
s1 = s1.sort_index()
c = [int(i) for i in x.split(':')]
s1.index = s1.index + pd.DateOffset(minutes=c[0]*60 + c[1])
pd.DataFrame(s1)

2010-03-02 10:30:00     82.70
2010-03-03 10:30:00    101.85
2010-03-04 10:30:00     53.90
2010-03-05 10:30:00     82.45
2010-03-06 10:30:00       NaN
                        ...  
2022-01-09 10:30:00       NaN
2022-01-10 10:30:00    332.25
2022-01-11 10:30:00    453.00
2022-01-12 10:30:00    246.85
2022-01-13 10:30:00    301.50
Freq: D, Length: 4336, dtype: float64

In [98]:
s1 = pd.DataFrame(s1,columns=['range_1030'])

In [100]:
pd.options.display.max_rows = 100

In [106]:
merge=pd.merge(base_df,s1, how='outer', left_index=True, right_index=True)
merge['range_1030'] = merge['range_1030'].fillna(method='ffill')
merge

,open,high,low,close,range_1030
2010-03-02 09:01:00,8826.90,8856.95,8826.90,8839.80,NaN
2010-03-02 09:02:00,8840.75,8853.90,8832.80,8852.30,NaN
2010-03-02 09:03:00,8855.70,8855.70,8844.65,8846.65,NaN
2010-03-02 09:04:00,8843.85,8850.70,8843.75,8846.70,NaN
2010-03-02 09:05:00,8846.40,8846.40,8837.75,8841.60,NaN
...,...,...,...,...,...
2022-01-13 15:28:00,38464.35,38469.15,38459.45,38467.20,301.5
2022-01-13 15:29:00,38469.00,38479.85,38456.65,38456.65,301.5
2022-01-13 15:30:00,38464.05,38471.55,38455.15,38465.70,301.5
2022-01-13 15:31:00,38466.05,38466.05,38466.05,38466.05,301.5


In [108]:
merge.between_time('10:29', '10:31')

,open,high,low,close,range_1030
2010-03-02 10:29:00,8864.05,8868.35,8863.60,8867.20,NaN
2010-03-02 10:30:00,8865.05,8868.80,8865.05,8868.25,82.70
2010-03-02 10:31:00,8867.70,8870.95,8867.70,8869.30,82.70
2010-03-03 10:29:00,9020.85,9020.85,9017.75,9018.55,82.70
2010-03-03 10:30:00,9020.10,9021.00,9019.05,9020.50,101.85
...,...,...,...,...,...
2022-01-12 10:30:00,38835.50,38841.40,38828.80,38840.60,246.85
2022-01-12 10:31:00,38838.20,38840.65,38818.35,38828.90,246.85
2022-01-13 10:29:00,38475.45,38478.15,38459.95,38464.00,246.85
2022-01-13 10:30:00,38461.60,38465.05,38455.30,38458.15,301.50


In [109]:
import datetime as DT
t1 = DT.datetime.strptime('00:05:36.0100000', '%H:%M:%S.%f0')
t2 = DT.datetime(1900,1,1)

print((t1-t2).total_seconds() / 60.0)

5.6001666666666665


In [122]:
x = '10:30'
c = [int(i) for i in x.split(':')]

In [124]:
c[0]*60 + c[1]

630

In [121]:
x.split(':')

['10', '30']